In [32]:
from utilities import *
from core.initial import Insertions
from core.solver import CGCPSolver
from docplex.cp.model import *
from docplex.mp.model import Model
import pandas as pd

In [2]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')
model = CSPModel(d)
model.build_model()
initial = Insertions(model)
initial.solve()
s = CGCPSolver(initial.sol)

In [ ]:
rmp = Model(name='Restricted_Master_Problem')

rmp.trips = s.trips
rmp.duties = s.duties

rmp.trip_duty_arr = rmp.binary_var_matrix(s.trips, s.duties)

In [ ]:
rmp.duty_arr = rmp.binary_var_list(rmp.duties, name='duty')

In [ ]:
for trip in rmp.trips:
    duty_ct = rmp.sum(rmp.trip_duty_arr[(trip, duty)] * rmp.duty_arr[duty.ID] for duty in rmp.duties) == 1
    rmp.add_constraint(duty_ct)

In [ ]:
rmp.minimize(rmp.sum(rmp.duty_arr[duty.ID] for duty in rmp.duties))

In [ ]:
rmp.solve()

In [34]:
sub = CpoModel(name="Pricing_Subproblem")

In [35]:
trips = [interval_var(start=trip.start_time, end=trip.end_time, length=trip.duration, size=trip.duration, name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]

In [41]:
ntrips = len(model.trips)

duties = [interval_var(length=model.constraints.shift_span, size=model.constraints.shift_span, name=f"Duty_{i}") for i in range(ntrips)]

In [44]:
duties[0].end

(0, 4503599627370494)

In [33]:
INTERVAL_MAX

4503599627370494